In [1]:
#libraries
import cv2
import numpy as np
import pandas as pd
from sklearn.neighbors import KDTree
import json
import xml.etree.ElementTree as ET
from tqdm import tqdm
import zipfile
import os

In [2]:
# read the .xml-file
# zip_ref = zipfile.ZipFile('../CompanyMen_v1.0-split-012-Bobby_being_angry.azp')
zip_ref = zipfile.ZipFile('her_scene11_fuerChristian.azp')
zip_ref.extractall('/tmp')
tree = ET.parse('/tmp/content.xml')
root = tree.getroot().findall('./{http://experience.univ-lyon1.fr/advene/ns}annotations')
i=0
ts_end=[]
ts_begin=[]
for child in root[0].iter():
    if child.get('type')=='#Shot':
        i+=1
        for child2 in child:
            if child2.tag=='{http://experience.univ-lyon1.fr/advene/ns}millisecond-fragment':
                ts_end.append(round(int(child2.get('end'))/1000*25))
                ts_begin.append(round(int(child2.get('begin'))/1000*25))

In [3]:
def read_target_colors_azp(azp_path):
    zip_ref = zipfile.ZipFile(azp_path)
    zip_ref.extractall('/tmp')
    tree = ET.parse('/tmp/content.xml')
    root = tree.getroot().findall('./{http://experience.univ-lyon1.fr/advene/ns}annotations')
    colors_target=[]
    for child in root[0].iter():
        if child.get('type')=='#ColourRange':
            for child2 in child:
                if child2.tag=='{http://experience.univ-lyon1.fr/advene/ns}content':
                    colors_target.append(child2.text.split(','))
    return colors_target

In [4]:
print('ts_begin: ',np.shape(ts_begin),ts_begin)
print('ts_end: ',np.shape(ts_end),ts_end)

ts_begin:  (23,) [0, 239, 465, 4436, 4510, 4602, 4651, 4717, 4813, 4930, 4974, 5017, 5247, 5278, 5546, 5643, 5684, 5730, 5795, 5846, 5953, 6008, 6043]
ts_end:  (23,) [239, 465, 4436, 4510, 4602, 4651, 4717, 4813, 4930, 4974, 5017, 5247, 5278, 5546, 5643, 5684, 5730, 5795, 5846, 5953, 6008, 6043, 6387]


In [5]:
#read video file frame by frame, beginning and ending with a timestamp
def read_video_segments(video,start_frame,end_frame,resolution_width,target_colorspace):
    resolution_height=int(round(resolution_width * 9/16))
    resolution=(resolution_width,resolution_height)
    vid = cv2.VideoCapture(video)
    frames=[]
    vid_length=0
    with tqdm(total=end_frame-start_frame+1) as pbar: #init the progressbar,with max lenght of the given segment
        while(vid.isOpened()):
            # Capture frame-by-frame
            ret, frame = vid.read() # if ret is false, frame has no content
            if not ret:
                break
            # skip every "skip_frame"
            if vid_length>=start_frame:
                # resize the video to a different resolution
                frame=cv2.resize(frame,resolution)
                frame=np.array(frame,dtype='uint8')
                frames.append(frame) #add the individual frames to a list
                pbar.update(1) #update the progressbar
            if vid_length==end_frame:
                pbar.update(1)
                break
            vid_length+=1 #increase the vid_length counter
    vid.release()
    cv2.destroyAllWindows()
    frames=change_colorspace(frames,target_colorspace)
    return frames[:-1]

In [6]:
def change_colorspace(frame_list,target_colorspace):
    changed_frame_list=[]
    if target_colorspace=='HSV':
        print('HSV')
        for frame in frame_list:
            changed_frame_list.append(cv2.cvtColor(frame, cv2.COLOR_BGR2HSV))
        return changed_frame_list
    if target_colorspace=='cie-lab':
        print('cie-lab')
        for frame in frame_list:
            changed_frame_list.append(cv2.cvtColor(frame, cv2.COLOR_BGR2LAB))
        return changed_frame_list
    else:
        print('rgb')
        for frame in frame_list:
            changed_frame_list.append(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        return changed_frame_list

In [7]:
def fn_rgb_to_color(target_colorspace,path):
            if (path != 'full'):
                print('Now using colors specified in path')
                colors = {}
                with open(path) as f:
                    for line in f:
                        #split lines at "::
                        color, rgb = line.strip().split(':')
                        #strip the rgb-string of the parenthesis, split it up a the commas,
                        #cast them to int and put them into a tuples
                        rgb_value=tuple(map(int,(rgb.strip('(').strip(')').split(','))))
                        colors[color]=rgb_value
            else:
                colors={'darkred':(139,0,0),
                'firebrick':(178,34,34),
                'crimson':(220,20,60),
                'red':(255,0,0),
                'tomato':(255,99,71),
                'salmon':(250,128,114),
                'darkorange':(255,140,0),
                'gold':(255,215,0),
                'darkkhaki':(189,183,107),
                'yellow':(255,255,0),
                'darkolivegreen':(85,107,47),
                'olivedrab':(107,142,35),
                'greenyellow':(173,255,47),
                'darkgreen':(0,100,0),
                'aquamarine':(127,255,212),
                'steelblue':(70,130,180),
                'skyblue':(135,206,235),
                'darkblue':(0,0,139),
                'blue':(0,0,255),
                'royalblue':(65,105,225),
                'purple':(128,0,128),
                'violet':(238,130,238),
                'deeppink':(255,20,147),
                'pink':(255,192,203),
                'antiquewhite':(250,235,215),
                'saddlebrown':(139,69,19),
                'sandybrown':(244,164,96),
                'ivory':(255,255,240),
                'dimgrey':(105,105,105),
                'grey':(128,128,128),
                'silver':(192,192,192),
                'lightgrey':(211,211,211),
                'black':(0,0,0),
                'white':(255,255,255),
                'darkcyan':(0,139,139),
                'cyan':(0,255,255),
                'green':(0,128,0),
                'khaki':(240,230,140),
                'goldenrod':(218,165,32),
                'orange':(255,165,0),
                'coral':(255,127,80),
                'magenta':(255,0,255),
                'wheat':(245,222,179),
                'skin':(255,224,189),
                'purple4':(147,112,219)}

            colors_aux={}
            if target_colorspace=='HSV':
                print('HSV')
                for color in colors:
                    a = np.array((colors[color]),dtype='uint8')
                    b = a.reshape(1,1,3)
                    c = cv2.cvtColor(b,cv2.COLOR_RGB2HSV)
                    colors_aux[color]=tuple(c.reshape(3))
                colors=colors_aux
            if target_colorspace=='cie-lab':
                print('cie-lab')
                for color in colors:
                    a = np.array((colors[color]),dtype='uint8')
                    b = a.reshape(1,1,3)
                    c = cv2.cvtColor(b,cv2.COLOR_RGB2LAB)
                    colors_aux[color]=tuple(c.reshape(3))
                colors=colors_aux

            rgb_to_color={}
            for color in colors:
                rgb_to_color[colors[color]]=color
            #purple4 is median purple
            #skin is caucasian        
            return rgb_to_color

In [8]:
def create_nns_picture(frame_list,target_colorspace,path):
    rgb_to_color=fn_rgb_to_color(target_colorspace,path) #get the color dict 
    bins={} #a dict with an entry for each for histograms 
    for rgb in rgb_to_color: #init the dict with zeros for every key
        bins[rgb_to_color[rgb]]=0
        
    rgb_list=[] #create a list of the rgb_values
    for rgb in rgb_to_color: #map the values of the dict to a list
        rgb_list.append(rgb)
    kdt = KDTree(rgb_list, leaf_size=30, metric='euclidean')  
    #flatten the image to 1d 
#     img = frame_list[0].reshape((frame_list[0].shape[0] * frame_list[0].shape[1], 3))
    img = frame_list.reshape((frame_list.shape[0] * frame_list.shape[1], 3))     

    nns = kdt.query(img, k=1, return_distance=False)
    changed_frame_aux=[]
    for nn in tqdm(nns):
        changed_frame_aux.append(rgb_list[nn[0]])
    changed_frame_aux=np.asarray(changed_frame_aux,dtype='uint8')
#     changed_frame=changed_frame_aux.reshape(frame_list[0].shape[0],frame_list[0].shape[1],3)
    changed_frame=changed_frame_aux.reshape(frame_list.shape[0],frame_list.shape[1],3)
    return changed_frame

In [38]:
resolution=200
target_colorspace='cie-lab'
color_path='colors'
# color_path='full'
# video_path='../../Wells_John_CompanyMen_full.mp4'
video_path='Her_bluray_Szene 11_25fps.mp4'
scene_all_frames_lab=[]
for i,ts in enumerate(zip(ts_begin,ts_end)):
    scene_all_frames_lab.append(read_video_segments(video_path,
                                                    ts[0],ts[1],resolution,target_colorspace))
    if i==5:
        break

full_scene_all_frames_lab=[]
for i,ts in enumerate(zip(ts_begin,ts_end)):
    full_scene_all_frames_lab.append(read_video_segments(video_path,
                                                    ts[0],ts[1],resolution,'full'))
    if i==5:
        break

241it [00:01, 150.43it/s]                         
  0%|          | 0/227 [00:00<?, ?it/s]

cie-lab


228it [00:02, 91.85it/s]                         
  0%|          | 0/3972 [00:00<?, ?it/s]

cie-lab


3973it [00:22, 173.60it/s]                          


...

rgb


94it [00:20,  4.48it/s]                        
  0%|          | 0/50 [00:00<?, ?it/s]

rgb


51it [00:20,  2.46it/s]                        

rgb


In [10]:
print('shape: ', np.shape(scene_all_frames_lab))

shape:  (6,)


In [11]:
# for i,frame_list in enumerate(scene_all_frames_lab):
# #     print('full',len(frame_list))
# #     print('half',int(len(frame_list)/2))
#     index=int(len(frame_list)/2)
#     try:
# #         frame=frame_list[index]
#         frame=create_nns_picture(frame_list[index],target_colorspace,color_path)
#         cv2.imwrite('key_frames/ganzer_film/lab_full_200w_'+str(i)+'.png',cv2.cvtColor(frame, cv2.COLOR_LAB2BGR))
#     except:
#         print('no valid timestamps at '+str(i))
#         pass

In [12]:
def extract_dominant_colors(frame_list,target_colorspace,path):
    print(str(len(frame_list))+' frames to process.')
    rgb_to_color=fn_rgb_to_color(target_colorspace,path) #get the color dict 
    bins={} #bins dict for histograms 
    for rgb in rgb_to_color: #init the dict with ones for every key to avoid difficulties with divisions
                            # because the the sum of the bins goes from 500k to 2kk this shouldn't be a problem
        bins[rgb_to_color[rgb]]=1
    rgb_list=[] #create a traverseable list of the rgb_values
    for rgb in rgb_to_color: #map the values of the dict to a list
        rgb_list.append(rgb)

    kdt = KDTree(rgb_list, leaf_size=30, metric='euclidean')
    for image in tqdm(frame_list): #traverse the video
        img = image.reshape((image.shape[0] * image.shape[1], 3)) #flatten the image to 1d   
        nns = kdt.query(img, k=1, return_distance=False)
        for nn in nns:
            bins[rgb_to_color[rgb_list[nn[0]]]]+=1
    norm_factor = len(frame_list)* np.shape(frame_list[0])[0] * np.shape(frame_list[0])[1] #normalize the bins
    bins_norm={k:v/norm_factor*10000 for k,v in bins.items()} #scale 0-10000 for visibility
    return bins_norm

In [13]:
def bins_to_df(bins,bin_threshold=5,colors_to_return=5):
    #create a dataframe
    bins_sorted=list(zip(list(bins.values()),list(bins.keys())))
    df=pd.DataFrame(bins_sorted,columns=['count','color'])
    df.set_index('color',inplace=True) #set the colors as the index of the dataframe
#     bin_threshold=bin_threshold/100 #scale the percentage to 0-1
#     df = df[df>bin_threshold].dropna() #kick bins from the dataframe with precentage lower than bin_threshold 
    return df

In [41]:
reduced_dataframes=[]
for shot in scene_all_frames_lab:
    bins=extract_dominant_colors(shot,target_colorspace,color_path)
    reduced_dataframes.append(bins_to_df(bins))
full_dataframes=[]
for shot in full_scene_all_frames_lab:
    bins=extract_dominant_colors(shot,target_colorspace,color_path)
    full_dataframes.append(bins_to_df(bins))

  2%|▏         | 5/239 [00:00<00:04, 49.52it/s]

239 frames to process.
Now using colors specified in path
cie-lab


  4%|▍         | 9/226 [00:00<00:02, 84.02it/s]

226 frames to process.
Now using colors specified in path
cie-lab


  0%|          | 7/3971 [00:00<00:56, 69.83it/s]

...

 10%|▉         | 9/92 [00:00<00:00, 87.63it/s]

92 frames to process.
Now using colors specified in path
cie-lab


 18%|█▊        | 9/49 [00:00<00:00, 82.90it/s]

49 frames to process.
Now using colors specified in path
cie-lab


100%|██████████| 49/49 [00:00<00:00, 82.15it/s]


In [42]:
print(len(reduced_dataframes),len(full_dataframes))

6 6


In [137]:
def process_df(reduced_dataframe_list,full_dataframe_list,ground_truth,noise_threshold):
    # create the lists that are returned
    # add the first entry of the histogramm to the output 
    # the 'real' output is the real_dataframe_list
    real_dataframe_list=[0]
    comparison=[0]
    
    # traverse the histograms
    # d is the current shot, d1 the following shot
    for i,(d,d1,e,e1) in enumerate(zip(reduced_dataframe_list,reduced_dataframe_list[1:],
                                       full_dataframe_list,full_dataframe_list[1:])):
        
        #calculate the absolute change of the histograms
        absolute_df=abs(d1-d) 
        absolute_df=absolute_df.sort_values(by='count',ascending=False)
        
        #apply a noise-filter to the absolute df
#         reduced_absolute_high=reduced_absolute_df[reduced_absolute_df>noise_threshold].dropna()
#         reduced_absolute_low=reduced_absolute_df[reduced_absolute_df<-noise_threshold].dropna()
#         reduced_absolute_denoised=reduced_absolute_high.combine_first(reduced_absolute_low)
        absolute_denoised=absolute_df[absolute_df>noise_threshold].dropna()
        
        # create the relative change, scaled to 0-100 percent
        # they are calculated from the denoised df, to prevent errors because of the noise
        relative_df=absolute_denoised/d*100
        relative_df=relative_df.dropna()
        relative_df=relative_df.sort_values(by='count',ascending=False)
        
        reduced_dd=d.sort_values(by='count',ascending=False)
        
        #use the relative changes as weights for the absolute dataframe
        weighted_absolute_df=absolute_df*relative_df
        weighted_absolute_df=weighted_absolute_df.sort_values(by='count',ascending=False)
        
#         if i>0:
#             print(reduced_absolute_df)
#             print(reduced_relative_df)
#             print(reduced_absolute_df*reduced_relative_df)
#             break

        # if there are no elements in the denoised df, it is assumed that all changes from
        # shot to shot are noise, in that case the current histogram is appended,
        # else the relative histogram
        if len(absolute_denoised)==0:
            real_dataframe_list.append(reduced_dd)
        else:
            real_dataframe_list.append(relative_df)
        
        # create a comparison table
        relative=relative_df.head()
        relative=relative.drop('count',axis=1)
        relative=relative.reset_index(level=0,inplace=False)
        relative=relative.rename(index=str,columns={'color':'relative'})
        
        new=weighted_absolute_df.head()
        new=new.drop('count',axis=1)
        new=new.reset_index(level=0,inplace=False)
        new=new.rename(index=str,columns={'color':'new'})
        
        reduced_old=reduced_dd.head()
        reduced_old=reduced_old.drop('count',axis=1)
        reduced_old=reduced_old.reset_index(level=0,inplace=False)
        reduced_old=reduced_old.rename(index=str,columns={'color':'reduced_old'})
        
        full_dd=e.sort_values(by='count',ascending=False)
        full_old=full_dd.head()
        full_old=full_old.drop('count',axis=1)
        full_old=full_old.reset_index(level=0,inplace=False)
        full_old=full_old.rename(index=str,columns={'color':'full_old'})

        gt=pd.DataFrame(ground_truth[i+1],columns=['ground_truth'])
        
        joined=pd.concat([full_old,reduced_old,new,relative,gt],axis=1)
        comparison.append(joined)
        
    return {'real': real_dataframe_list,'comparison':comparison}

In [138]:
ground_truth=read_target_colors_azp('her_scene11_fuerChristian.azp')
new = process_df(reduced_dataframes,full_dataframes,ground_truth,100)

In [139]:
# new['comparison'][0]

In [140]:
new['comparison'][1]

,full_old,reduced_old,new,relative,ground_truth
0,black,black,black,NaN,NaN
1,dark_cyan,brown,brown,NaN,NaN
2,grey,grey,grey,NaN,NaN
3,red,red,red,NaN,NaN
4,dark_orange,dark_orange,dark_orange,NaN,NaN
0,NaN,NaN,NaN,NaN,black
1,NaN,NaN,NaN,NaN,dimgrey
2,NaN,NaN,NaN,NaN,grey


In [141]:
new['comparison'][2]

,full_old,reduced_old,new,relative,ground_truth
0,black,black,grey,grey,NaN
1,dark_cyan,brown,black,black,NaN
2,grey,grey,brown,NaN,NaN
3,royal_blue,red,dark_cyan,NaN,NaN
4,red,dark_orange,dark_orange,NaN,NaN
0,NaN,NaN,NaN,NaN,black
1,NaN,NaN,NaN,NaN,dimgrey
2,NaN,NaN,NaN,NaN,grey


In [142]:
new['comparison'][3]

,full_old,reduced_old,new,relative,ground_truth
0,black,black,brown,white,NaN
1,dark_cyan,grey,grey,brown,NaN
2,grey,brown,white,grey,NaN
3,royal_blue,dark_cyan,black,black,NaN
4,white,dark_orange,dark_cyan,NaN,NaN
0,NaN,NaN,NaN,NaN,wheat1
1,NaN,NaN,NaN,NaN,black
2,NaN,NaN,NaN,NaN,orange
3,NaN,NaN,NaN,NaN,salmon
4,NaN,NaN,NaN,NaN,silver


In [143]:
new['comparison'][4]

,full_old,reduced_old,new,relative,ground_truth
0,dark_cyan,grey,dark_orange,dark_orange,NaN
1,grey,brown,white,white,NaN
2,black,black,grey,black,NaN
3,royal_blue,white,black,grey,NaN
4,dark_orange,dark_orange,brown,brown,NaN
0,NaN,NaN,NaN,NaN,coral
1,NaN,NaN,NaN,NaN,darkcyan
2,NaN,NaN,NaN,NaN,yellow
3,NaN,NaN,NaN,NaN,olivedrab


In [144]:
new['comparison'][5]

,full_old,reduced_old,new,relative,ground_truth
0,dark_cyan,grey,black,black,NaN
1,grey,brown,grey,white,NaN
2,black,black,brown,dark_orange,NaN
3,royal_blue,white,white,grey,NaN
4,dark_orange,dark_orange,dark_orange,brown,NaN
0,NaN,NaN,NaN,NaN,white
1,NaN,NaN,NaN,NaN,yellow
2,NaN,NaN,NaN,NaN,skin


In [37]:
# new['comparison'][6]